In [1]:
from typing import List, Dict, Tuple, Callable, Type, Union, Optional, ClassVar
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd
from sko.GA import GA
from scipy.spatial.distance import pdist, squareform
import plotly.graph_objects as go

### Class to run multimodal optimization

In [2]:
class MultiModalOptimizer(ABC):
    def __init__(
        self,
        objective_func: Callable[[np.ndarray], np.ndarray],
        n_dim: int,
        batch_size: int,
        bounds: Optional[List[Tuple[float, float]]] =None,
        pop_size: int =100,
        generations: int =500
    ):
        self.objective_func = objective_func
        self.n_dim = n_dim
        self.batch_size = batch_size
        self.bounds = bounds if bounds is not None else [(0, 1)]*n_dim
        self.pop_size = pop_size
        self.generations = generations

        # Multimodal progress record attributes
        self._mmo_score_names = [
            'pop_id', 'objective', 'diversity',
            'nomral_objective', 'normal_diversity', 'mmo_loss'
        ]
        self._mmo_scores: Dict[str, List[float]] = {
            k: [] for k in self._mmo_score_names
        }

    def _scalar_objective_func(self, *args) -> float:
        x = np.array(args)
        return self.objective_func(x.reshape(1, -1)).item()

    def run_mmo(self) -> Tuple[np.ndarray, pd.DataFrame]:

        # Clean multimodal progress record
        self._mmo_scores: Dict[str, List[float]] = {
            k: [] for k in self._mmo_score_names
        }

        # Run multi-objective optimization and return promising explored samples
        X = self.minimize()

        # Perform the core multi-modal optimization task
        # by selecting diverse optima from the candidate solutions
        X_batch, df_mmo_scores = self.select_diverse_optima(X)

        # self.print_results(X_batch, df_mmo_scores)

        return X_batch, df_mmo_scores

    def minimize(self):
        # Initialize and run GA for minimization
        ga = GA(
            func=self._scalar_objective_func,
            n_dim=self.n_dim,
            size_pop=self.pop_size,
            max_iter=self.generations,
            prob_mut=0.2,
            lb=[val[0] for val in self.bounds],
            ub=[val[1] for val in self.bounds],
        )
        best_x, best_y = ga.run()
        
        # Get the last generation population
        population = ga.X
        return population

    def select_diverse_optima(self, X: np.ndarray) -> Tuple[np.ndarray, pd.DataFrame]:
        """
        Select diversified batch of candidates among explored points during
        minimization.
        """
        X = np.atleast_2d(X)

        # Get objective values of the optimization explored points
        objective_loss = self.objective_func(X)

        # Normalize objective values to [0, 1] range
        normalized_objective = self._normalize(objective_loss)

        # Calculate pairwise distances between individuals
        distances = squareform(pdist(X))

        # Initialize selected candidates with the best sample
        first_index = np.argmin(objective_loss)  # best sample with smallest objective value
        selected_indices = [first_index]

        # Initialize list of not yet selected indices
        n_samples = X.shape[0]
        available_indices = list(range(n_samples))
        available_indices.remove(first_index)

        # Set dict to store selection progress
        self._store_mmo_progress(
            pop_id=first_index,
            objective=objective_loss[first_index],
            diversity=np.nan,
            nomral_objective=normalized_objective[first_index],
            normal_diversity=np.nan,
            mmo_loss=normalized_objective[first_index],
        )

        # Select diverse candidates
        for _ in range(1, min(self.batch_size, n_samples)):

            # Calculate diversity score: mean of distances to already selected candidates
            diversity_score = distances[:, selected_indices].mean(axis=1)

            # Normalize diversity scores to [0, 1] range
            normalized_diversity = self._normalize(diversity_score)

            # Combine normalized objective and diversity scores
            # objective is minimized while diversity is maximized
            combined_loss = normalized_objective - normalized_diversity  # This is the main operation

            # Remove rows of already selected indices 
            available_combined_loss = [combined_loss[i] for i in available_indices]

            # Select the individual with the smallest loss
            next_index = available_indices[np.argmin(available_combined_loss)]

            selected_indices.append(next_index)
            available_indices.remove(next_index)

            # Store selection progress
            self._store_mmo_progress(
                pop_id=next_index,
                objective=objective_loss[next_index],
                diversity=diversity_score[next_index],
                nomral_objective=normalized_objective[next_index],
                normal_diversity=normalized_diversity[next_index],
                mmo_loss=combined_loss[next_index],
            )

        return X[selected_indices], pd.DataFrame(self._mmo_scores)

    def _normalize(self, values: np.ndarray) -> np.ndarray:
        """Normalize values to [0, 1] range."""
        min_val, max_val = values.min(), values.max()
        return (values - min_val) / (max_val - min_val) if max_val > min_val else np.zeros_like(values)

    def _store_mmo_progress(self, **kwargs):
        # Store selection progress
        for key in self._mmo_score_names:
            if key not in kwargs.keys():
                raise KeyError(f"Multimodal selection progess storing is missing '{key}'")

            self._mmo_scores[key].append(kwargs[key])

    def print_results(self, X_batch: np.ndarray, df_mmo_scores: pd.DataFrame):
        feature_cols = [f'feature_{i+1}' for i in range(X_batch.shape[1])]
        df_candidates = pd.DataFrame(X_batch, columns=feature_cols)
        df_print = pd.concat(
            [df_candidates, df_mmo_scores],
            axis=1, ignore_index=False
        )
        print(
            f"{self.__class__.__name__} -> Selected points to be input to "
            f"the simulator: \n{df_print}"
        )

### Main Objective function to minimize (Loss function)

In [65]:
def shubert_like_function(X) -> np.ndarray:
    """
    A multi-dimensional function based on the provided equation.
    This function has multiple local minima.
    """
    X = 2*(X - 0.5)  # Transform [0, 1]^p into [-1, 1]^p
    return np.mean(0.5 * ((1 - np.abs(X)) * np.cos(10 * np.pi * X**2) + 1), axis=1)

n_dim = 2  # You can change this to any number of dimensions
bounds = [(0, 1)]*n_dim  # Adjusted bounds to capture interesting regions of the function

### Plot function

In [66]:
plot_range = np.array([
    [bounds[0][0], bounds[0][1]],
    [bounds[1][0], bounds[1][1]]
])

# Create a grid of points
xx, yy = np.meshgrid(
    np.arange(plot_range[0, 0], plot_range[0, 1], 0.01),
    np.arange(plot_range[1, 0], plot_range[1, 1], 0.01),
)
grid_points = np.c_[xx.ravel(), yy.ravel()]

# Set Colorbar to use for plots
colorbar = dict(
    len=0.9,
    thickness=30,
    title_text='Loss',
    title_side='bottom',
    y=0., yanchor='bottom',  # Anchor point for vertical positioning
    x=1.0, xanchor='left',  # Anchor point for horizontal positioning
)

# Loss function values
Z = shubert_like_function(grid_points)
Z = Z.reshape(xx.shape)

# Create the heatmap with countours
loss_heatmap = go.Heatmap(
    x=xx[0], 
    y=yy[:, 0],
    z=Z,
    colorscale='viridis',
    colorbar=colorbar,
)

# Set aspect ratio to get same scale on x and y axis
x_range = plot_range[0, 1] - plot_range[0, 0]
y_range = plot_range[1, 1] - plot_range[1, 0]
aspect_ratio = y_range / x_range
fig_size = 700

# Set ticks on integers
x_ticks = np.arange(np.ceil(plot_range[0, 0]), np.floor(plot_range[0, 1]) + 1, 1)
y_ticks = np.arange(np.ceil(plot_range[1, 0]), np.floor(plot_range[1, 1]) + 1, 1)

# Set figure layout
layout_dict = dict(
    xaxis_title="X1",
    yaxis_title="X2",
    showlegend=True,
    width=fig_size,
    height=fig_size*aspect_ratio,
    xaxis=dict(
        range=plot_range[0, :],
        tickmode='array',
        tickvals=x_ticks,
        ticktext=x_ticks.astype(int)
    ),
    yaxis=dict(
        range=plot_range[1, :],
        tickmode='array',
        tickvals=y_ticks,
        ticktext=y_ticks.astype(int)
    ),
)

# Visualize the results (for 2D case)
def plot_results_2d(X_batch, loss_values, optim_config):
    # Set scatter trace for batch
    trace_batch = go.Scatter(
        x=X_batch[:, 0],
        y=X_batch[:, 1],
        mode='markers',
        name='Batch',
        marker=dict(
            color='red',
            size=10,
            symbol='circle',
            line_width=1
        ),
        text=[f'rank: {i+1}, loss: {loss:.4f}' for i, loss in enumerate(loss_values)],
        hoverinfo='text'
    )

    fig = go.Figure(
        data=[loss_heatmap, trace_batch],
        layout=dict(
            title=f"Shubert-like Function minimized with {optim_config}",
            **layout_dict
        )
    )
    fig.show()

### Case 1

In [67]:
# Set up the optimization
batch_size = 8
pop_size = 100
generations = 100

optimizer = MultiModalOptimizer(shubert_like_function, n_dim=n_dim, batch_size=batch_size, bounds=bounds, pop_size=pop_size, generations=generations)
X_batch, df_mmo_scores = optimizer.run_mmo()
print(df_mmo_scores.iloc[:, 1:].describe())

# If n_dim is 2, plot the results
if n_dim == 2:
    plot_results_2d(X_batch, df_mmo_scores['objective'], {'pop_size': pop_size, 'generations': generations})
elif n_dim > 2:
    print("Visualization is only available for 2D. Current dimension:", n_dim)

       objective  diversity  nomral_objective  normal_diversity  mmo_loss
count   8.000000   7.000000          8.000000          7.000000  8.000000
mean    0.388763   0.743594          0.277935          0.937681 -0.542536
std     0.111785   0.053903          0.142698          0.065306  0.229964
min     0.171038   0.652364          0.000000          0.846882 -0.742053
25%     0.351157   0.714287          0.229930          0.886980 -0.652816
50%     0.410702   0.756552          0.305942          0.942928 -0.585433
75%     0.461949   0.779313          0.371360          1.000000 -0.558543
max     0.508955   0.809041          0.431366          1.000000  0.000000


### Case 2

In [68]:
# Set up the optimization
batch_size = 8
pop_size = 50
generations = 200

optimizer = MultiModalOptimizer(shubert_like_function, n_dim=n_dim, batch_size=batch_size, bounds=bounds, pop_size=pop_size, generations=generations)
X_batch, df_mmo_scores = optimizer.run_mmo()
print(df_mmo_scores.iloc[:, 1:].describe())

# If n_dim is 2, plot the results
if n_dim == 2:
    plot_results_2d(X_batch, df_mmo_scores['objective'], {'pop_size': pop_size, 'generations': generations})
elif n_dim > 2:
    print("Visualization is only available for 2D. Current dimension:", n_dim)

       objective  diversity  nomral_objective  normal_diversity  mmo_loss
count   8.000000   7.000000          8.000000          7.000000  8.000000
mean    0.383606   0.759303          0.391912          0.932736 -0.424232
std     0.099722   0.045582          0.188037          0.114736  0.206051
min     0.175763   0.693363          0.000000          0.696373 -0.702224
25%     0.343307   0.729201          0.315924          0.916389 -0.538135
50%     0.415333   0.777233          0.451737          1.000000 -0.425089
75%     0.440329   0.779784          0.498870          1.000000 -0.373432
max     0.501162   0.826557          0.613578          1.000000  0.000000


### Case 3

In [69]:
# Set up the optimization
batch_size = 8
pop_size = 20
generations = 500

optimizer = MultiModalOptimizer(shubert_like_function, n_dim=n_dim, batch_size=batch_size, bounds=bounds, pop_size=pop_size, generations=generations)
X_batch, df_mmo_scores = optimizer.run_mmo()
print(df_mmo_scores.iloc[:, 1:].describe())

# If n_dim is 2, plot the results
if n_dim == 2:
    plot_results_2d(X_batch, df_mmo_scores['objective'], {'pop_size': pop_size, 'generations': generations})
elif n_dim > 2:
    print("Visualization is only available for 2D. Current dimension:", n_dim)

       objective  diversity  nomral_objective  normal_diversity  mmo_loss
count   8.000000   7.000000          8.000000          7.000000  8.000000
mean    0.378218   0.602918          0.177073          0.857396 -0.573148
std     0.091097   0.067755          0.152517          0.205112  0.270864
min     0.272454   0.513390          0.000000          0.482010 -0.897357
25%     0.320357   0.550044          0.080200          0.759880 -0.732246
50%     0.359517   0.605984          0.145762          1.000000 -0.611158
75%     0.429282   0.653445          0.262565          1.000000 -0.498250
max     0.525111   0.694075          0.423004          1.000000  0.000000


Run 3 GA experiments all with 10k total evaluations where (pop_size, generations) in [(100, 100), (50, 200), (20, 500)]. And they seem all good. Can't decide which one is better and what is the effect of pop_size or generations params.